In [10]:
import torch 
from torch import nn
import numpy as np
class ReviewClassifier(nn.Module):
    def __init__(self):
        super().__init__()        
        self.mlp = nn.Sequential(
            nn.Linear(384, 150),
            nn.ReLU(),
            nn.Linear(150,15),
            nn.ReLU(),
            nn.Linear(15, 1),
            nn.Sigmoid()
        ) 
        
    
    def forward(self, x):        
        return self.mlp(x)

In [11]:
state_dict = torch.load("review_classifier.bin")

In [12]:
import pandas as pd 
test_df = pd.read_feather("test_df.bin")

In [13]:
model = ReviewClassifier()
model.load_state_dict(state_dict)

<All keys matched successfully>

In [20]:
N = 28
test_example = torch.Tensor(np.array(test_df.embeddings[N]))
model.forward(test_example)


tensor([0.9626], grad_fn=<SigmoidBackward0>)

In [21]:
test_df.content[N]

'There are very few albums where every song is one that I want to listen to over and over. Clay\'s voice is beautiful and he should have many hits from this CD. "Touch" is amazing.'

In [9]:
test_df.label[N]

1

In [10]:
from torchmetrics import Accuracy 

from tqdm import tqdm 
model.eval()
metric = Accuracy(task="binary")
for _, test_row in tqdm(test_df.iterrows(), desc='evaluating'):
    pred = model.forward(
        torch.Tensor(np.array(test_row.embeddings))
    )
    metric.update(pred, torch.Tensor([test_row.label]))
metric_value = metric.compute()

In [17]:
# from tqdm import tqdm 
# model.eval()
# metric = Accuracy(task="binary")
# for _, test_row in tqdm(test_df.iterrows(), desc='evaluating'):
#     pred = model.forward(
#         torch.Tensor(np.array(test_row.embeddings))
#     )
#     metric.update(pred, torch.Tensor([test_row.label]))
# metric_value = metric.compute()

evaluating: 6000it [00:02, 2529.11it/s]


In [18]:
metric_value

tensor(0.9515)